# Creating population grid

In [ ]:
# TODO: Get a pop hex at same resolution as existing grid
# NOTE: check that this is not too small compared to cell size of pop grid?

In [1]:
import h3
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import json
import pickle
from rasterio.plot import show
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rioxarray as rxr
from shapely.geometry import Polygon

In [2]:
%run ../settings/yaml_variables.py
%run ../settings/df_styler.py
%run ../settings/plotting.py
%run ../settings/load_osmdata.py
% run ../settings/paths.py

h3_res = h3.h3_get_resolution(osm_grid.loc[0,'hex_id'])
print(f"Using H3 resolution {h3_res} for calculating population density")

<string>:49: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
/Users/anev/Library/CloudStorage/Dropbox/ITU/repositories/bikedna_dk_analysis/settings/plotting.py:49: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap_name, n)


OSM data loaded successfully!
Using H3 resolution 8 for calculating population density


In [ ]:
# LOAD DATA
pop_src_1 = rasterio.open(pop_fp_1)
pop_src_2 = rasterio.open(pop_fp_2)

In [ ]:


# MERGE RASTERS
mosaic, out_trans = merge([pop_src_1, pop_src_2])

out_meta = pop_src_1.meta.copy()

out_meta.update(
    {
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": pop_src_1.crs,
    }
)
merged_fp = "../data/intermediary/pop/merged_pop_raster.tif"
with rasterio.open(merged_fp, "w", **out_meta) as dest:
    dest.write(mosaic)

merged = rasterio.open(merged_fp)

In [ ]:


# Load DK boundaries
engine = dbf.connect_alc(db_name, db_user, db_password, db_port=db_port)

get_muni = "SELECT navn, kommunekode, geometry FROM muni_boundaries"

muni = gpd.GeoDataFrame.from_postgis(get_muni, engine, geom_col="geometry")

dissolved = muni.dissolve()
dissolved_buffer = dissolved.buffer(500)

dissolved_proj = dissolved_buffer.to_crs(merged.crs)
convex = dissolved_proj.convex_hull

geo = gpd.GeoDataFrame({"geometry": convex[0]}, index=[0], crs=merged.crs)

coords = [json.loads(geo.to_json())["features"][0]["geometry"]]

# Clip raster to DK extent
clipped, out_transform = mask(merged, shapes=coords, crop=True)

out_meta = merged.meta.copy()

out_meta.update(
    {
        "driver": "GTiff",
        "height": clipped.shape[1],
        "width": clipped.shape[2],
        "transform": out_transform,
        "crs": merged.crs,
    }
)
clipped_fp = "../data/intermediary/pop/clipped_pop_raster.tif"
with rasterio.open(clipped_fp, "w", **out_meta) as dest:
    dest.write(clipped)


# RESAMPLE TO GRID SIZE OF 400 meters

downscale_factor = 1 / 4

with rasterio.open(clipped_fp) as dataset:

    # resample data to target shape
    resampled = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * downscale_factor),
            int(dataset.width * downscale_factor),
        ),
        resampling=Resampling.bilinear,
    )

    # scale image transform
    out_transform = dataset.transform * dataset.transform.scale(
        (dataset.width / resampled.shape[-1]), (dataset.height / resampled.shape[-2])
    )

out_meta.update(
    {
        "driver": "GTiff",
        "height": resampled.shape[1],
        "width": resampled.shape[2],
        "transform": out_transform,
        "crs": merged.crs,
    }
)

resamp_fp = "../data/intermediary/pop/resampled_pop_raster.tif"
with rasterio.open(resamp_fp, "w", **out_meta) as dest:
    dest.write(resampled)

test = rasterio.open(resamp_fp)
assert round(test.res[0]) == 400
assert round(test.res[1]) == 400


# REPROJECT TO CRS USED BY H3
dst_crs = "EPSG:4326"
proj_fp_wgs84 = "../data/intermediary/pop/reproj_pop_raster_wgs84.tif"

with rasterio.open(resamp_fp) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    kwargs = src.meta.copy()
    kwargs.update(
        {"crs": dst_crs, "transform": transform, "width": width, "height": height}
    )

    with rasterio.open(proj_fp_wgs84, "w", **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear,
            )


test = rasterio.open(proj_fp_wgs84)
assert test.crs.to_string() == "EPSG:4326"

print("Population data has been merged, clipped, reprojected and downsampled!")

# COMBINE WITH H3 DATA
pop_df = (
    rxr.open_rasterio(proj_fp_wgs84)
    .sel(band=1)
    .to_pandas()
    .stack()
    .reset_index()
    .rename(columns={"x": "lng", "y": "lat", 0: "population"})
)

# Ignore no data values
pop_df = pop_df[pop_df.population > -200]

pop_gdf = gpd.GeoDataFrame(pop_df, geometry=gpd.points_from_xy(pop_df.lng, pop_df.lat))

pop_gdf.set_crs("EPSG:4326", inplace=True)

dk_gdf = gpd.GeoDataFrame({"geometry": dissolved_proj}, crs=dissolved_proj.crs)
dk_gdf.to_crs("EPSG:4326", inplace=True)

pop_gdf = gpd.sjoin(pop_gdf, dk_gdf, op="within", how="inner")

pf.plot_scatter(pop_gdf, metric_col="population", marker=".", colormap="Oranges")


# INDEX POPULATION AT VARIOUS H3 LEVELS
for res in range(6, 11):
    col_hex_id = "hex_id_{}".format(res)
    col_geom = "geometry_{}".format(res)
    msg_ = "At resolution {} -->  H3 cell id : {} and its geometry: {} "
    print(msg_.format(res, col_hex_id, col_geom))

    pop_gdf[col_hex_id] = pop_gdf.apply(
        lambda row: h3.geo_to_h3(lat=row["lat"], lng=row["lng"], resolution=res), axis=1
    )

    # use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    pop_gdf[col_geom] = pop_gdf[col_hex_id].apply(
        lambda x: {
            "type": "Polygon",
            "coordinates": [h3.h3_to_geo_boundary(h=x, geo_json=True)],
        }
    )

#%%
# Test plot
hex_id_col = "hex_id_8"
h3_groups = (
    pop_gdf.groupby(hex_id_col)["population"].sum().to_frame("population").reset_index()
)

h3_groups["lat"] = h3_groups[hex_id_col].apply(lambda x: h3.h3_to_geo(x)[0])
h3_groups["lng"] = h3_groups[hex_id_col].apply(lambda x: h3.h3_to_geo(x)[1])

h3_groups["hex_geometry"] = h3_groups[hex_id_col].apply(
    lambda x: {
        "type": "Polygon",
        "coordinates": [h3.h3_to_geo_boundary(h=x, geo_json=True)],
    }
)

h3_groups.plot.scatter(
    x="lng",
    y="lat",
    c="population",
    marker="o",
    edgecolors="none",
    colormap="Oranges",
    figsize=(30, 20),
)
plt.xticks([], [])
plt.yticks([], [])
plt.title("hex-grid: population")

h3_pop_level = 7

# Convert to H3 polygons
print(f"Creating hexagons at resolution {h3_pop_level}...")

hex_id_col = f"hex_id_{h3_pop_level}"

h3_groups = (
    pop_gdf.groupby(hex_id_col)["population"].sum().to_frame("population").reset_index()
)

h3_groups["lat"] = h3_groups[hex_id_col].apply(lambda x: h3.h3_to_geo(x)[0])
h3_groups["lng"] = h3_groups[hex_id_col].apply(lambda x: h3.h3_to_geo(x)[1])

h3_groups["hex_geometry"] = h3_groups[hex_id_col].apply(
    lambda x: {
        "type": "Polygon",
        "coordinates": [h3.h3_to_geo_boundary(h=x, geo_json=True)],
    }
)

h3_groups["geometry"] = h3_groups["hex_geometry"].apply(
    lambda x: Polygon(list(x["coordinates"][0]))
)

h3_gdf = gpd.GeoDataFrame(h3_groups, geometry="geometry", crs="EPSG:4326")

h3_gdf.to_crs(crs, inplace=True)



In [ ]:
# Export data
h3_gdf.to_file(f"../data/intermediary/pop/h3_pop_{h3_pop_level}_polygons.gpkg")
